# Do it 자연어 처리 8장 문장 생성
## 8.1 문장 생성 모델 훑어보기
### 문장 생성
- 컨텍스트가 주어졌을 때 다음 단어로 어떤 단어가 오는 게 적절한지 분류
### 문장 생성 방식
1. 컨텍스트를 모델에 입력해 다음 토큰 확률을 출력한 뒤 다음 토큰 선택
2. 기존 컨텍스트에 1에서 선택한 다음 토큰을 이어붙인 새로운 컨텍스트를 모셀에 입력해 다음 토큰 확률 분포를 추출하고 그 다음 토큰을 선택
3. 2 반복
---




## 8.2 문장 생성 모델 파인튜닝

### .1-2 각종 설정

In [16]:
#!pip install ratsnlp

In [17]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [18]:
# 모델 환경 설정
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=1,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [19]:
# 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [20]:
# 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cp

### .3 말뭉치 내려받기

In [21]:
# 말뭉치 내려받기
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[Korpora] Corpus `nsmc` is already installed at /content/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /content/Korpora/nsmc/ratings_test.txt


### .4 토크나이저 준비

In [22]:
# 토크나이저 준비
# eos_token은 문장 마지막에 붙이는 스페셜 토큰으로 sk텔레콤이 모델을 프리트레인할 때 이렇게 지정했기 때문에 같은 방식으로 사용
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### .5 데이터 전처리하기

In [23]:
# 학습 데이터셋 구축
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 3.263 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 3.263 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 3.263 s]


문장 앞에 극성 정보를 부여함 -> 해당 극성에 맞는 문장을 생성할 수 있는, 조건부 문장 생성 능력이 있는지를 검증하고자 함

In [24]:
train_dataset[0]

GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [25]:
# 학습 데이터 로더 구축
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [26]:
# 평가용 데이터 로더 구축
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 0.549 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 0.549 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_PreTrainedTokenizerFast_32_nsmc_sentence-generation [took 0.549 s]


### .6 프리트레인 마친 모델 읽어들이기

In [27]:
# 모델 초기화
# 프리트레인이 완료된 GPT2 모델을 읽고, 문장 생성 모델을 초기화
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

### .7 모델 파인튜닝

In [28]:
# 태스크 정의
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

In [29]:
# 트레이너 정의
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
# 학습
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-generation exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size 

Training: 0it [00:00, ?it/s]

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 244, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 289, in _run
    self._record_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/record_writer.py", line 43, in flush
    self._writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 221, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.FailedPreconditionError: /gdrive/My Drive/nlpbook/checkpoint-generation/lightning_logs/version_0/events.out.tfevents.1709465843.83746c4784bc.558.0; Transport endpoint is not connected


Validation: 0it [00:00, ?it/s]